In [ ]:
import cv2 as cv
import numpy as np
from scipy.interpolate import griddata
from matplotlib import pyplot as plt
from scipy import ndimage

In [ ]:
SIZE_OF_CHUNK = 1000

In [ ]:
data = np.load("Data/CropConcatData.npy")
mask = ~np.isnan(data)

In [ ]:
plt.imshow(data)

In [ ]:
#Select the "iterations" value so that the holes inside the map disappear
ITERATIONS = 3

In [ ]:
struct = ndimage.generate_binary_structure(2, 2)
mask = ndimage.binary_dilation(mask, struct, iterations=ITERATIONS)
mask = ndimage.binary_erosion(mask, struct, iterations=ITERATIONS)

In [ ]:

plt.imshow(mask)


### Interpolate

In [ ]:
rows, cols = data.shape

rowsData = np.array_split(data, range(SIZE_OF_CHUNK, rows, SIZE_OF_CHUNK), axis=0)
rowsMask = np.array_split(mask, range(SIZE_OF_CHUNK, rows, SIZE_OF_CHUNK), axis=0)
   
for rowData, rowMask in zip(rowsData, rowsMask):
    colsData = np.array_split(rowData, range(SIZE_OF_CHUNK, cols, SIZE_OF_CHUNK), axis=1)
    colsMask = np.array_split(rowMask, range(SIZE_OF_CHUNK, cols, SIZE_OF_CHUNK), axis=1)
    for colData, colMask, in zip(colsData, colsMask):
        clipMask =  np.logical_and(np.isnan(colData), colMask)
        nan_indices = np.where(clipMask == True)
        not_nan_indices = np.argwhere(~np.isnan(colData))
        not_nan_values = colData[~np.isnan(colData)]
        if nan_indices[0].shape[0] != 0:

            if len(not_nan_indices) != 0:
                colData[nan_indices] = griddata(not_nan_indices, not_nan_values, nan_indices, method='linear' )
    rowData= np.concatenate(colsData, 1)
data = np.concatenate(rowsData)

### Save data

In [ ]:
np.save("Data/InterpolatedData.npy", data)

In [ ]:
data = np.nan_to_num(data, nan=0)
plt.imsave("Results/InterpolatedMap.png", data)